In [23]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [24]:
loader = TextLoader("../data/Tenacious.txt", encoding="windows-1252")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

# loader = TextLoader("../data/Robinson_Advisory.txt", encoding="windows-1252")
# index = VectorstoreIndexCreator().from_loaders([loader])

# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=1024, chunk_overlap=0
# )
# documents = loader.load_and_split()
# docs = text_splitter.split_documents(documents)
# embeddings = OpenAIEmbeddings()

Created a chunk of size 1343, which is longer than the specified 1000


In [25]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [27]:
weaviate_client = weaviate.connect_to_local()
db = WeaviateVectorStore.from_documents(docs, embeddings, client=weaviate_client)

In [28]:
query = "What is this week challenge?"
docs = db.similarity_search(query)

# Print the first 100 characters of each result
for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:100] + "...")


Document 1:
One of the most dangerous thoughts for a technical person is to make an
uninformed or emotion based ...

Document 2:
The solution should focus on specific students tackling specific challenges, such
as taking universi...

Document 3:
gv Tenacious
Intelligence

Corporation

Tenacious Talent - GenAl
UpSkilling

GEN-AI Challenge
TEAM-M...

Document 4:
3. Architectures:
e Explore different agent architectures, such as rule-based,
learning-based, and h...


In [29]:
with open("../data/Tenacious.txt", encoding="windows-1252") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

Created a chunk of size 1343, which is longer than the specified 1000


In [30]:
docsearch = WeaviateVectorStore.from_texts(
    texts,
    embeddings,
    client=weaviate_client,
    metadatas=[{"source": f"{i}-pl"} for i in range(len(texts))],
)

retriever = docsearch.as_retriever()

In [34]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: context\nAnswer:\n"))]


In [35]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is this week's challenge?")

"I'm sorry, I don't know the answer to that question."